# **Addons**

Addons is a feature of dicfg that can be added to the config dictionary keys via ```@addon_type{addon_name}``` syntax.
Currently there are these types of addons:
- validator (@validate(...))
- updater (@update(...))
- templates (@validate(...))

Below these types are discussed in more detail.


In [5]:
from pprint import pprint

from dicfg import ConfigReader

## Validators: use validators with **@validate(validator-name)** in config files

Dicfg has the following builtin validators: 
 - **required**: using this validator throws an error if the value is empty, useful when a value is required to be set by a user.
 - **depreciated**: using this validator throws an error if the values is not empty, useful when a name has changed and should'nt be used anymore
 - **object**: using this validator throws an error if the object is not defined with "*object", or if object can not be loaded, or if arguments are not valid arguments. 

In [27]:
print("Main config:\n")
!cat ./configs/config_with_validators.yml
config_reader = ConfigReader(name="myconfig", main_config_path="./configs/config_with_validators.yml")
user_config = {'myconfig': {'default': {'config_int2': 3}}}
print("\n"+"Output config:\n")
pprint(config_reader.read(user_config), sort_dicts=False)  

Main config:

default:
    config_int2@validator(required): 
    config_int@validator(depreciated): 
    object:
      "*object": "io.StringIO"
    config_string: "fire"
    config_list: [1, 2, 3]
    config_none: None
    config_dict:
        sub_config: "water"


        
Output config:

{'default': {'config_int2': 3,
             'config_int': None,
             'object': {'*object': 'io.StringIO'},
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'}}}


## Updaters: use updates with **@update(updater-name)** in config files

Dicfg has the following builtin validators: 
 - **replace**: using this updater replaces a list or dict completely
 - **merge**: using this updater merges a list or dict with the additional values

**Note**: Default behaviour or a dict is merge and the default behaviour of a list is replace

In [28]:
print("Main config:\n")
!cat ./configs/config_with_updaters.yml
config_reader = ConfigReader(name="myconfig", main_config_path="./configs/config_with_updaters.yml")
user_config = {'myconfig': {'default': {'config_list': [20], 'config_dict': {'new': 'new'}}}}
print("\n"+"Output config:\n")
pprint(config_reader.read(user_config), sort_dicts=False)  

Main config:

default:
    config_string: "fire"
    config_list@updater(merge): [1, 2, 3]
    config_none: None
    config_dict@updater(replace):
        sub_config: "water"


        
Output config:

{'default': {'config_string': 'fire',
             'config_list': [1, 2, 3, 20],
             'config_none': 'None',
             'config_dict': {'new': 'new'}}}


## Templates: use templates with **@template(template-name)** in config files

Templates are predefined configs defined in subclass of TemplateAddon. Here below is an example with StringIO:

In [13]:
!cat /Users/mart/code/dicfg/dicfg/addons/templates.py


from dicfg.addons.addons import TemplateAddon


class StringIOTemplate(TemplateAddon):
    """Template for io.StringIO object"""

    NAME = "stringio"

    @property
    def data(self):
        return {
            "*object": "io.StringIO",
            "initial_value@validator(required)": "",
            "newline": 3,
        }


This template can be used as follows:

In [29]:
print("Main config:\n")
!cat ./configs/config_with_templates.yml
config_reader = ConfigReader(name="myconfig", main_config_path="./configs/config_with_templates.yml")
user_config = {'myconfig': {'default': {'mystringio': {'initial_value': 'hello_template'}}}}
print("\n"+"Output config:\n")
pprint(config_reader.read(user_config), sort_dicts=False)  

Main config:

default:
    config_string: "fire"
    config_list: [1, 2, 3]
    config_none: None
    config_dict:
        sub_config: "water"
    mystringio@template(stringio):

        
Output config:

{'default': {'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'mystringio': {'*object': 'io.StringIO',
                            'initial_value': 'hello_template',
                            'newline': 3}}}
